In [1]:
library(xgboost)
library(tidyverse)

Warning message:
"package 'xgboost' was built under R version 3.6.3"-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.1     v dplyr   1.0.5
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'readr' was built under R version 3.6.3"Warning message:
"package 'purrr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"Warning message:
"package 'forcats' was built under R version 3.6.3"-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x 

In [2]:
data <- read_csv("src/animal_diseases/data.csv")


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  .default = col_character(),
  Id = col_double(),
  latitude = col_double(),
  longitude = col_double(),
  sumAtRisk = col_double(),
  sumCases = col_double(),
  sumDeaths = col_double(),
  sumDestroyed = col_double(),
  sumSlaughtered = col_double(),
  humansAge = col_double(),
  humansAffected = col_double(),
  humansDeaths = col_double()
)
i Use `spec()` for the full column specifications.



In [3]:
set.seed(42)
data <- data[sample(1:nrow(data)), ]

In [4]:
head(data)

Id,source,latitude,longitude,region,country,admin1,localityName,localityQuality,observationDate,...,speciesDescription,sumAtRisk,sumCases,sumDeaths,sumDestroyed,sumSlaughtered,humansGenderDesc,humansAge,humansAffected,humansDeaths
217744,OIE,46.26800,5.065800,Europe,France,Rhone-Alpes,CONFRANCON,Unknown,30/09/2016,...,"domestic, cattle",40,2,0,0,0,NA,NA,NA,NA
226906,National authorities,39.01444,100.665833,Asia,China,Gansu Sheng,Zhangye,Unknown,13/04/2017,...,NA,NA,NA,NA,NA,NA,NA,NA,1,NA
207828,OIE,14.73860,107.608600,Asia,Viet Nam,Kon Tum,Dak Xu,Exact,04/10/2015,...,"domestic, unspecified bird",2701,2685,1500,1201,0,NA,NA,NA,NA
218094,OIE,47.64087,31.905330,Europe,Ukraine,Mykolayivs'ka,Kalynivka,Exact,13/10/2016,...,"domestic, swine",90,4,4,86,0,NA,NA,NA,NA
215678,OIE,48.34070,31.235900,Europe,Ukraine,Kirovohrads'ka,Khmelyove,Exact,14/08/2016,...,"domestic, swine",NA,1,1,0,0,NA,NA,NA,NA
210658,OIE,47.97034,3.255003,Europe,France,Bourgogne,PRECY-SUR-VRIN,Exact,10/02/2016,...,"domestic, cattle",258,1,0,0,0,NA,NA,NA,NA


In [5]:
train_data <- data %>% select(-starts_with("human"))

In [6]:
train_label <- data %>% select(humansAffected) %>% is.na() %>% magrittr::not()

In [7]:
head(train_label)

humansAffected
FALSE
TRUE
FALSE
FALSE
FALSE
FALSE


In [8]:
train_data_only_numeric <- train_data %>% select(-Id) %>% select(-c(latitude, longitude)) %>% select_if(is.numeric)

In [9]:
str(train_data_only_numeric)

tibble[,5] [17,008 x 5] (S3: tbl_df/tbl/data.frame)
 $ sumAtRisk     : num [1:17008] 40 NA 2701 90 NA ...
 $ sumCases      : num [1:17008] 2 NA 2685 4 1 ...
 $ sumDeaths     : num [1:17008] 0 NA 1500 4 1 ...
 $ sumDestroyed  : num [1:17008] 0 NA 1201 86 0 ...
 $ sumSlaughtered: num [1:17008] 0 NA 0 0 0 0 0 465 0 NA ...


In [10]:
region <- model.matrix(~country-1, data)

In [11]:
ncol(region)

[1] 122

In [12]:
head(region)

countryAlbania,countryAlgeria,countryAngola,countryArgentina,countryArmenia,countryAustria,countryBahrain,countryBangladesh,countryBelgium,countryBelize,...,countryUganda,countryUkraine,countryUnited Arab Emirates,countryUnited Republic of Tanzania,countryUnited States of America,countryUruguay,countryViet Nam,countryWest Bank,countryZambia,countryZimbabwe
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
head(data$speciesDescription)

[1] "domestic, cattle"           NA                          
[3] "domestic, unspecified bird" "domestic, swine"           
[5] "domestic, swine"            "domestic, cattle"

In [14]:
train_data_only_numeric$is_domestic <- str_detect(data$speciesDescription, "domestic")

In [15]:
head(train_data_only_numeric$is_domestic)

[1] TRUE   NA TRUE TRUE TRUE TRUE

In [16]:
species_list <- data$speciesDescription %>% str_replace("[[:punct:]]", "") %>% str_extract("[a-z]*$")

In [17]:
speciesList <- tibble(species = species_list)

In [18]:
options(na.action='na.pass')
species <- model.matrix(~species-1, speciesList)

In [19]:
head(species)

species,speciesanatidae,speciesbird,speciesboar,speciesbuffaloe,speciesbuzzard,speciescamelidae,speciescanine,speciescat,speciescattle,...,speciessheep,speciesstarling,speciesstork,speciesswan,speciesswine,speciesteal,speciesturkey,specieswigeon,speciesxpeacock,speciesyaks
0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_data_only_numeric <- cbind(train_data_only_numeric, region, species)
data_matrix <- data.matrix(train_data_only_numeric)

In [21]:
head(data_matrix)

sumAtRisk,sumCases,sumDeaths,sumDestroyed,sumSlaughtered,is_domestic,countryAlbania,countryAlgeria,countryAngola,countryArgentina,...,speciessheep,speciesstarling,speciesstork,speciesswan,speciesswine,speciesteal,speciesturkey,specieswigeon,speciesxpeacock,speciesyaks
40,2,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NA,NA,NA,NA,NA,NA,0,0,0,0,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2701,2685,1500,1201,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,4,4,86,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
NA,1,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
258,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
split <- round(length(train_label)*0.7)

In [23]:
train_data <- data_matrix[1:split,]
train_labels <- train_label[1:split]

test_data <- data_matrix[-(1:split), ]
test_labels <- train_label[-(1:split)]

In [24]:
dtrain <- xgb.DMatrix(train_data, label = train_labels)
dtest <- xgb.DMatrix(test_data, label = test_labels)

In [25]:
eta <- .2
max_depth <- 3
nround <- 10
neg = sum(train_labels == FALSE)
pos = sum(train_labels == TRUE)

In [26]:
model <- xgboost(data = dtrain,
                eta = eta,
                max_depth = max_depth,
                objective = "binary:logistic",
                scale_pos_weight = neg/pos,
                nround = nround,
                early_stopping_rounds = 3,
                eval_metric = 'error'
                )

[1]	train-error:0.016042 
Will train until train_error hasn't improved in 3 rounds.

[2]	train-error:0.015370 
[3]	train-error:0.015370 
[4]	train-error:0.014698 
[5]	train-error:0.014698 
[6]	train-error:0.014614 
[7]	train-error:0.014698 
[8]	train-error:0.014698 
[9]	train-error:0.014698 
Stopping. Best iteration:
[6]	train-error:0.014614



In [27]:
pred <- predict(model, dtest)

err <- mean(as.numeric(pred > 0.5) != test_labels)
print(paste("test-error=", err))

[1] "test-error= 0.0137201097608781"
